In [ ]:
import pandas as pd
from IPython.display import display
import os
import re
import Bio
from Bio import Entrez
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency


In [ ]:
'''
In this notebook I am going to get the data used for the species count donut chart and the phylum cluster map in the 
supplementary.

'''

In [ ]:
#this is modified from my data fetcher to add scientific names to my csv columns

tax_LOT = pd.read_pickle(r"C:\PATH\ncbi_2025_taxonomy_table.pkl")
print(tax_LOT)
email = 'example@gmail.com' 


def fetch_TAXID_data(tax_id, email):
    classification = tax_id
    Entrez.email = email  
    retry_limit = 3  

    attempts = 0
    while attempts < retry_limit:
        try:
            handle = Entrez.efetch(db="taxonomy", id=str(classification), retmode="xml")
            records = Entrez.read(handle)
            for rec in records:
                NAME = rec.get("ScientificName")
            break  
        except Exception as e:
            print(f"HTTP error: {str(e)} - Sleeping 10s and retrying for ID {classification}")
            sleep(10)
            attempts += 1  
        finally:
            if 'handle' in locals() and handle:
                handle.close()  
    if attempts == retry_limit:
        print(f"Failed after {retry_limit} retries for ID {classification}")

    return NAME



def fetch_LINEAGE_NAME_data(lineage_list, email):
    Entrez.email = email 
    lineage_df = {}
    
    for classification in lineage_list:  # same as first instance
        try:
            NAME_series = tax_LOT[tax_LOT['tax_id'] == classification]['name_txt']
            if not NAME_series.empty:
                NAME = NAME_series.iloc[0]  
                if NAME:
                    lineage_df[NAME] = classification
                    print(f'NAME: {NAME}, Tax ID: {classification}')
                else:
                    print(f'No rank available for Tax ID: {classification}')
            else:
                print(f'No entry found for Tax ID: {classification}, fetching from NCBI...')
                NAME = fetch_TAXID_data(classification, email)
                if NAME and NAME != "No NAME available":
                    lineage_df[NAME] = classification
                else:
                    print(f'Failed to fetch data for Tax ID: {classification} from NCBI.')
        except Exception as e:
            print(f'Error processing Tax ID {classification}: {str(e)}')

    return lineage_df

# testing everything still works (0 fails):
list_tax = ['1', '0', '2624677', '1644055', '2157', '1236']
email = 'your_email@example.com'  # Replace with your actual email
print(fetch_LINEAGE_NAME_data(list_tax, email))

In [ ]:
#load the false positive filtered species data
species_summary_df = pd.read_excel(r"C:\PATH\5_Filtered_FBDS_IPRcounts_byspecies_MAUA_names.xlsx")

In [ ]:
species_summary_df2 = species_summary_df.fillna(0)
print(species_summary_df2)
#species_summary_df2 = species_summary_df2.drop('IPR026259', axis = 1)
#print(species_summary_df2)


In [ ]:
#IPR dataframe masks:
#getting counts of IPR combos

total_species = species_summary_df2['species'].count()

IPR009908_only = (species_summary_df2['IPR009908'] > 0) & (species_summary_df2['IPR003752'] == 0) & (species_summary_df2['IPR012932'] == 0)

IPR012932_only = (species_summary_df2['IPR009908'] == 0) & (species_summary_df2['IPR003752'] == 0) & (species_summary_df2['IPR012932'] > 0)

IPR003752_only = (species_summary_df2['IPR009908'] == 0) & (species_summary_df2['IPR003752'] > 0) & (species_summary_df2['IPR012932'] == 0)

IPR003752_IPR009908_only = (species_summary_df2['IPR009908'] > 0) & (species_summary_df2['IPR003752'] > 0) & (species_summary_df2['IPR012932'] == 0)

IPR003752_IPR012932 = (species_summary_df2['IPR012932'] > 0) & (species_summary_df2['IPR003752'] > 0) & (species_summary_df2['IPR009908'] == 0)

IPR009908_IPR012932_only = (species_summary_df2['IPR009908'] > 0) & (species_summary_df2['IPR012932'] > 0)  & (species_summary_df2['IPR003752'] == 0)

all_three = (species_summary_df2['IPR009908'] > 0) & (species_summary_df2['IPR012932'] > 0) & (species_summary_df2['IPR003752'] > 0)
none = (species_summary_df2['IPR009908'] == 0) & (species_summary_df2['IPR012932'] == 0) & (species_summary_df2['IPR003752'] == 0)


In [ ]:
print(total_species)
print(IPR009908_only)


In [ ]:
print(species_summary_df2.groupby('phylum')['species'].count())

In [ ]:
print(species_summary_df2)

#some masking to get the groups
phylum_total = species_summary_df2.groupby('phylum')['species'].nunique()
phylum_ipr009908 = species_summary_df2.loc[IPR009908_only].groupby('phylum')['species'].nunique()
phylum_ipr012932 = species_summary_df2.loc[IPR012932_only].groupby('phylum')['species'].nunique()
phylum_ipr003752 = species_summary_df2.loc[IPR003752_only].groupby('phylum')['species'].nunique()


phylum_all_three = species_summary_df2.loc[all_three].groupby('phylum')['species'].nunique()
phylum_ipr009908_ipr012932 = species_summary_df2.loc[IPR009908_IPR012932_only].groupby('phylum')['species'].nunique()
phylum_ipr012932_ipr003752 = species_summary_df2.loc[IPR003752_IPR012932].groupby('phylum')['species'].nunique()
phylum_ipr003752_ipr009908 = species_summary_df2.loc[IPR003752_IPR009908_only].groupby('phylum')['species'].nunique()

phylum_none = species_summary_df2.loc[none].groupby('phylum')['species'].nunique()


#making a new df
phylum_summary_df = pd.DataFrame({
    'total_count': phylum_total,
    'IPR009908_only': phylum_ipr009908,
    'IPR012932_only': phylum_ipr012932,
    'IPR003752_only': phylum_ipr003752,
    'IPR012932_IPR009908_only': phylum_ipr009908_ipr012932,
    'IPR003752__IPR012932_only': phylum_ipr012932_ipr003752,
    'IPR009908_IPR003752_only': phylum_ipr003752_ipr009908,

    'All_three': phylum_all_three,
    'None': phylum_none
  
})


phylum_summary_df.reset_index(inplace=True)
phylum_summary_df2 = phylum_summary_df.fillna(0)

#check it
display(phylum_summary_df2)

In [ ]:
#header_df = list(phylum_summary_df2.keys())
    
#phylum_summary_df2.to_csv(path_or_buf=r"C:\PATH\full_filtered_set_IPR_counts_per_phylum_only.csv",sep=",",header = header_df)
#phylum_summary_df2.to_excel(r"C:\PATH\6_FBDS_phylum_species_counts_nomauG_data_with3qualitymarkers.xlsx")

#run this if your just loading in the phylum summary
phylum_summary_df2 = pd.read_excel(r"C:\PATH\6_FBDS_phylum_species_counts_nomauG_data_with3qualitymarkers.xlsx")

In [ ]:
phylum_summary_df3 = phylum_summary_df2.copy()
IPR009908_only_list =[]
IPR003752_only_list=[]
IPR012932_only_list=[]
IPR009908_IPR003752_only_list=[]
IPR012932_IPR003752_only_list=[]
IPR012932_IPR009908_only_list=[]
all_three=[]
none=[]

for ind in phylum_summary_df3.index:
    percent_IPR009908 = (phylum_summary_df3['IPR009908_only'][ind]/phylum_summary_df3['total_count'][ind])*100
    percent_IPR003752 = (phylum_summary_df3['IPR003752_only'][ind]/phylum_summary_df3['total_count'][ind])*100
    percent_IPR012932 = (phylum_summary_df3['IPR012932_only'][ind]/phylum_summary_df3['total_count'][ind])*100
    percent_IPR012932_IPR003752 = (phylum_summary_df3['IPR003752__IPR012932_only'][ind]/phylum_summary_df3['total_count'][ind])*100
    percent_IPR009908_IPR003752 = (phylum_summary_df3['IPR009908_IPR003752_only'][ind]/phylum_summary_df3['total_count'][ind])*100
    percent_IPR009908_IPR012932 = (phylum_summary_df3['IPR012932_IPR009908_only'][ind]/phylum_summary_df3['total_count'][ind])*100
    percent_all_three = (phylum_summary_df3['All_three'][ind]/phylum_summary_df3['total_count'][ind])*100
    percent_none = (phylum_summary_df3['None'][ind]/phylum_summary_df3['total_count'][ind])*100

    IPR009908_only_list.append(percent_IPR009908)
    IPR003752_only_list.append(percent_IPR003752)
    IPR012932_only_list.append(percent_IPR012932)
    IPR009908_IPR003752_only_list.append(percent_IPR009908_IPR003752)
    IPR012932_IPR003752_only_list.append(percent_IPR012932_IPR003752)
    IPR012932_IPR009908_only_list.append(percent_IPR009908_IPR012932)
    all_three.append(percent_all_three)
    none.append(percent_none)
print(none)  
phylum_summary_df3['perc_IPR009908'] = IPR009908_only_list
phylum_summary_df3['perc_IPR012932']= IPR012932_only_list
phylum_summary_df3['perc_IPR003752']= IPR003752_only_list
phylum_summary_df3['perc_IPR009908_IPR003752']= IPR009908_IPR003752_only_list
phylum_summary_df3['perc_IPR012932_IPR003752']= IPR012932_IPR003752_only_list
phylum_summary_df3['perc_IPR012932_IPR009908']= IPR012932_IPR009908_only_list
phylum_summary_df3['perc_all_three']= all_three
phylum_summary_df3['perc_none']= none
display(phylum_summary_df3)


In [ ]:
#this section is prone to failing because of NCBI http errors. so I doubled it up and put try and except loops
#just manually fill any gaps left over. for my data set it pulled all 167 rows but it took a while. 
#it just adds the scientific names instead of id
phylum_summary_df4 = phylum_summary_df3
names = []
for ind in phylum_summary_df4.index:
    if phylum_summary_df4['phylum'][ind] == 0:
        name = 'no_phylum'
        names.append(name)
    else:
        try:
            tax_id = phylum_summary_df4['phylum'][ind]
            list_ind = [int(tax_id)] #I made the function accept a list which was good for the Interpro pull but annoying here 
            print(list_ind)
            name_df = fetch_LINEAGE_NAME_data(list_ind,email)
            temp_name_ls = list(name_df.keys())
            print(temp_name_ls[0])
            names.append(temp_name_ls[0])
        except:
            try:
                tax_id = phylum_summary_df4['phylum'][ind]
                list_ind = [int(tax_id)] #I made the function accept a list which was good for the Interpro pull but annoying here 
                print(list_ind)
                name_df = fetch_LINEAGE_NAME_data(list_ind,email)
                temp_name_ls = list(name_df.keys())
                print(temp_name_ls[0])
                names.append(temp_name_ls[0])
            except:
                continue
        
                
print(names)       
#phylum_summary_df4['scientific_names'] = 

In [ ]:
#small check
print(len(names))

In [ ]:
#adding in the names because I lost them at the phylum condensing step
phylum_summary_df4['scientific_names']=names
display(phylum_summary_df4)

In [ ]:
#exporting phylum data uncomment lines if you want csv
#header_df = list(phylum_summary_df4.keys())
#phylum_summary_df4.to_csv(path_or_buf=r"C:\PATH\filtered_IPR_counts_percentages_per_phylum_NO_phy_removed.csv",sep=",",header = header_df)

#phylum_summary_df4.to_excel(r"C:\PATH\7_phylum_iprs_with_perc_and_names_no_maug_three_quality_markers.xlsx")

#read excel if you are starting here
phylum_summary_df4 = pd.read_excel(r"C:\PATH\7_phylum_iprs_with_perc_and_names_no_maug_three_quality_markers.xlsx")


In [ ]:
#creating a df with only percentages
phylum_percentages=phylum_summary_df4[['perc_IPR009908', 'perc_IPR012932', 'perc_IPR003752', 'perc_IPR009908_IPR003752', 'perc_IPR012932_IPR003752', 'perc_IPR012932_IPR009908', 'perc_all_three','perc_none', 'scientific_names']].copy()

In [ ]:
#removing candidatus phylums
phylum_no_candidatus = phylum_summary_df4.copy()
# Remove rows containing 'candidatus' in 'scientific_names'
phylum_no_candidatus = phylum_no_candidatus[~phylum_no_candidatus['scientific_names'].str.contains('candidatus', case=False, na=False)]
display(phylum_no_candidatus)

In [ ]:
phylum_percentages2=phylum_no_candidatus[['perc_IPR009908', 'perc_IPR012932', 'perc_IPR003752', 'perc_IPR009908_IPR003752', 'perc_IPR012932_IPR003752', 'perc_IPR012932_IPR009908', 'perc_all_three','perc_none', 'scientific_names']].copy()

In [ ]:
#removing candidatus and no phylum or candidate so its just the concrete phylum. Some still remain after the first removal
phylum_no_candidatus2 = phylum_no_candidatus[~phylum_no_candidatus['scientific_names'].str.contains('candidatus', case=False, na=False)]
phylum_no_candidatus3 = phylum_no_candidatus2[~phylum_no_candidatus2['scientific_names'].str.contains('no_phylum', case=False, na=False)]
phylum_no_candidatus4 = phylum_no_candidatus3[~phylum_no_candidatus3['scientific_names'].str.contains('candidate', case=False, na=False)]
phylum_no_candidatus4.set_index('scientific_names', inplace=True)
phylum_no_candidatus5 = phylum_no_candidatus4.copy()
phylum_no_candidatus6 = phylum_no_candidatus5[
    (phylum_no_candidatus5['total_count'] >= 10)
]


In [ ]:
'''
if your running the first time then export the data set otherwise import the data set and set the index to scientific names 
so you can reproduce the clustermap figure and the donut chart

'''
#phylum_no_candidatus6.to_excel("C:\PATH\8_phylum_percentages_cand_removed_no_maug_three_quality_markers.xlsx")
#read if starting here, reset index too
phylum_no_candidatus6 = pd.read_excel(r"C:\PATH\8_phylum_percentages_cand_removed_no_maug_three_quality_markers.xlsx")
phylum_no_candidatus6 = phylum_no_candidatus6.set_index('scientific_names')

In [ ]:
display(phylum_no_candidatus6)


In [ ]:
#making the clustermap
# modify this to take or leave any percentage column
phylum_no_candidatus7=phylum_no_candidatus6[['perc_IPR009908', 'perc_IPR012932', 'perc_IPR003752', 'perc_IPR009908_IPR003752', 'perc_IPR012932_IPR003752', 'perc_IPR012932_IPR009908', 'perc_all_three' ]].copy()
g = sns.clustermap(phylum_no_candidatus7,figsize=(15,15),cmap='rocket')


In [ ]:
g.savefig(r"C:\PATH\red_IPRS_perc_noMaug_no_none_clustermap_3qualitymarkers.png", dpi='figure',  bbox_inches='tight' )


In [ ]:
print(phylum_no_candidatus6.columns)

In [ ]:
#making the donut chart
phylum_no_candidatus6 = pd.read_excel(r"C:\PATH\8_phylum_percentages_cand_removed_no_maug_three_quality_markers.xlsx")

# summing for the donut
maue = int(phylum_no_candidatus6['IPR009908_only'].sum()) + int(phylum_no_candidatus6['IPR012932_IPR009908_only'].sum()) + int(phylum_no_candidatus6['IPR009908_IPR003752_only'].sum()) + int(phylum_no_candidatus6['All_three'].sum())
vkor = int(phylum_no_candidatus6['IPR012932_only'].sum()) + int(phylum_no_candidatus6['IPR012932_IPR009908_only'].sum()) + int(phylum_no_candidatus6['IPR003752__IPR012932_only'].sum()) + int(phylum_no_candidatus6['All_three'].sum())
dsbb = int(phylum_no_candidatus6['IPR003752_only'].sum()) + int(phylum_no_candidatus6['IPR003752__IPR012932_only'].sum()) + int(phylum_no_candidatus6['IPR009908_IPR003752_only'].sum()) + int(phylum_no_candidatus6['All_three'].sum())
none = int(phylum_no_candidatus6['None'].sum())
total = int(phylum_no_candidatus6['total_count'].sum())

# Data and labels
data2 = [maue, dsbb, vkor, none]
print(data2,total)
names = ['MauE', 'DsbB', 'VKOR', 'None']
print(data2)

# Explosion and colors, for some reason the diameters are a little off so I tweeked them to be roughly same
explode = (0.05,0,0.05,0)
colors = sns.color_palette("flare",n_colors=8)

# this is a dumb way to assign the labels based on a function that determines whether the label matches the count 
#but I didnt want to start from scratch so here we are
def func(pct, all_values):
    total = sum(all_values)
    absolute = int(round(pct * total / 100.0))
    for i, value in enumerate(all_values):
        if value == absolute:
            return f"{value}"
    return ""

# Plotting 
plt.figure(figsize=(12, 12))  # Increase the chart size
plt.pie(data2, colors=colors, autopct=lambda pct: func(pct, data2),
        pctdistance=0.8, labels=names, explode=explode, textprops={'fontsize': 10, 'color': 'black'})  # Set label font size here

# Add center circle for the vibes
centre_circle = plt.Circle((0, 0), 0.50, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.show()

fig.savefig(r'C:\PATH\LIGHTFLARE_IPRS_donut_chart3markers.png', dpi='figure',  bbox_inches='tight', transparent=True )
